To calculate FST, I need to use unfolded SAF and SFS using the reference to polarize the alleles. I need a saf file for each population. first make filelists for each pop:


In [ ]:
 find . -name "*_pairedplus_marked_dupes.bam" -exec cp -i {} ../allBam/ \; 

In [ ]:
sed 's/$/_pairedplus_marked_dupes.bam/g' /local/home/sarahgw/WGS/Sequences/NJ_E/NJ_E_list>NJ_E_filelist

Calculate unfolded SAF for each population, see https://github.com/ANGSD/angsd/issues/65 for justification of folded vs unfolded 

## using only mappable regions

In [ ]:
for POP in NJ_U NY_U VT_U; do echo $POP;/local/home/sarahgw/angsd/angsd/angsd -b /local/home/sarahgw/WGS/Sequences/filelists/"$POP".filelist -anc /local/home/sarahgw/references/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -rf /local/home/sarahgw/WGS/QC/All_mylu_Map1.txt -out /local/home/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_unfilt; done

In [ ]:
for POP in NJ_E NY_E VT_E; do echo $POP;/local/home/sarahgw/angsd/angsd/angsd -b /local/home/sarahgw/WGS/Sequences/filelists/"$POP".filelist -anc /local/home/sarahgw/references/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -rf /local/home/sarahgw/WGS/QC/All_mylu_Map1.txt -out /local/home/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_unfilt; done

In [ ]:
for POP in VT_U_noOut NY_U_noOut; do echo $POP; /local/home/sarahgw/angsd/angsd -b /data/sarahgw/WGS/Sequences/filelists/"$POP".filelist -anc /local/home/sarahgw/references/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out /data/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_un; done

In [ ]:
/local/home/sarahgw/angsd/misc/realSFS check /data/sarahgw/WGS/SNPS/All_mylu/saf/VT_E_noOut_unfilt.saf.idx

In [ ]:
'GL430790' pos[4383]:4496 vs posd[4383-1]:4496
'GL430555' pos[79008]:93606 vs posd[79008-1]:93606

In [14]:
bear_file = open("/data/sarahgw/WGS/QC/All_mylu_Map1.txt.bear")
bear_file_out = open("/data/sarahgw/WGS/QC/All_mylu_Map1.txt.bear_redone", "w")

ct = 0

line = bear_file.readline()
idx = -10
while line:
        
    next_line = bear_file.readline()
    if not next_line:
        break
    #ct += 1
    #if ct == 15:
    #    break
    #print line
    #print next_line

    if '-' in next_line:
        bear_file_out.write(line)
        line = next_line
        idx = int(line.split("-")[-1])
        continue
    else:
        next_idx = next_line.split(":")[-1]

        if idx + 1 == int(next_idx):
            newline = line.split("-")[0] + "-" + next_idx
            bear_file_out.write(newline)       
        else:
            bear_file_out.write(line)
            bear_file_out.write(next_line)
        
        line = bear_file.readline()
        idx = -10

bear_file_out.close()      
        

        

Calculate 2d sfs for each pair 

need to limit to 1 billion sites, then take sum of each column

In [ ]:
for POP in NJ NY VT; do echo $POP; /local/home/sarahgw/angsd/angsd/misc/realSFS -rf /local/home/sarahgw/WGS/QC/All_mylu_Map1.txt /local/home/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_E_unfilt.saf.idx /local/home/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_U_unfilt.saf.idx  -nSites 1000000000 > /local/home/sarahgw/WGS/SNPS/All_mylu/sfs/"$POP"_EU_unfilt.ml; done

In [ ]:
for POP in NY VT; do echo $POP; /local/home/sarahgw/angsd/misc/realSFS /data/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_E_un.saf.idx /data/sarahgw/WGS/SNPS/All_mylu/saf/"$POP"_U_noOut_un.saf.idx -nSites 500000000 > /data/sarahgw/WGS/SNPS/All_mylu/sfs/"$POP"_EU_noOut_un.ml; done

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' /local/home/sarahgw/WGS/SNPS/All_mylu/sfs/NJ_EU_unfilt.ml  > /local/home/sarahgw/WGS/SNPS/All_mylu/sfs/NJ_EU_unfilt_sum.ml 

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/NY_E_un.saf.idx ~/WGS/saf/NY_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NY_EU_unfull

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' /local/home/sarahgw/WGS/SNPS/All_mylu/sfs/NY_EU_unfilt.ml > /local/home/sarahgw/WGS/SNPS/All_mylu/sfs/NY_EU_unfilt_sum.ml

In [ ]:
~/angsd/misc/realSFS -P 30 ~/WGS/SNPS/All_mylu/saf/VT_E_un.saf.idx ~/WGS/SNPS/All_mylu/saf/VT_U_un.saf.idx -nSites 100000000 > ~/WGS/SNPS/All_mylu/sfs/VT_EU_unfullnose.ml; echo "NY FST Done" | mailx gw.sarah@gmail.com

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/SNPS/All_mylu/sfs/VT_EU_unfilt.ml > ~/WGS/SNPS/All_mylu/sfs/VT_EU_unfilt_sum.ml

In [ ]:
#~/angsd/angsd/misc/realSFS -P 30 ~/WGS/saf/NJ_U_D_un.saf.idx ~/WGS/saf/NJ_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJ_EUD_unfull

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/NJ_EUD_unfull > ~/WGS/sfs/NJ_EUD_unfull_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 20 ~/WGS/saf/NJ_U_un.saf.idx ~/WGS/saf/NY_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJNY_U_unfullnose.ml

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/NJNY_U_unfullnose.ml > ~/WGS/sfs/NJNY_U_unfullnose_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 30 ~/WGS/saf/VT_U_un.saf.idx ~/WGS/saf/NY_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/VTNY_U_unfullnose.ml

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/VTNY_U_unfullnose.ml > ~/WGS/sfs/VTNY_U_unfullnose_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 30 ~/WGS/saf/NJ_U_un.saf.idx ~/WGS/saf/VT_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJVT_U_unfull

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/VTNJ_U_unfullnose.ml > ~/WGS/sfs/NJVT_U_unfullnose_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 30 ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/NY_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJNY_E_unfull

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/SNPS/All_mylu/sfs/NJNY_E_unfull > ~/WGS/SNPS/All_mylu/sfs/NJNY_E_unfull_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 10 ~/WGS/saf/VT_E_un.saf.idx ~/WGS/saf/NY_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/VTNY_E_unfull

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/VTNY_E_unfull > ~/WGS/sfs/VTNY_E_unfull_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS -P 30 ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/VT_E_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/NJVT_E_unfull

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/NJVT_E_unfull > ~/WGS/sfs/NJVT_E_unfull_sum.ml

Run a test to see if the FST value is close to zero when comparing the same populations

In [ ]:
~/angsd/angsd/misc/realSFS -P 30 ~/WGS/saf/VT_U_un.saf.idx ~/WGS/saf/VT_U_un.saf.idx -nSites 1000000000 > ~/WGS/sfs/VTVT_U_test.ml

In [ ]:
awk 'BEGIN{FS=OFS=" "}
     NR>=1{for (i=1;i<=NF;i++) a[i]+=$i}
     END{for (i=1;i<=NF;i++) printf a[i] OFS; printf "\n"}' ~/WGS/sfs/VTVT_U_test.ml > ~/WGS/sfs/VTVT_U_test_sum.ml

In [ ]:
~/angsd/angsd/misc/realSFS fst index ~/WGS/saf/VT_U_un.saf.idx ~/WGS/saf/VT_U_un.saf.idx -sfs ~/WGS/sfs/VTVT_U_test_sum.ml -fstout ~/WGS/sfs/VTVT_U_test

[sarahgw@annotate sfs]$ ~/angsd/angsd/misc/realSFS fst stats VTVT_U_test.fst.idx 
        -> Assuming idxname:VTVT_U_test.fst.idx
        -> Assuming .fst.gz file: VTVT_U_test.fst.gz
        -> FST.Unweight[nObs:1789195645]:-0.012657 Fst.Weight:-0.012658
-0.012657 -0.012658

Calculate FST for each pair

NY Exposed vs Unexposed:

In [ ]:
/local/home/sarahgw/angsd/angsd/misc/realSFS fst index ~/WGS/SNPS/All_mylu/saf/NY_E_unfilt.saf.idx ~/WGS/SNPS/All_mylu/saf/NY_U_unfilt.saf.idx -sfs ~/WGS/SNPS/All_mylu/sfs/NY_EU_unfilt_sum.ml -fstout NY_EU_unfilt

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NY_EU_unfilt.fst.idx

Fst.Weight:0.016614

In [ ]:
~/angsd/angsd/misc/realSFS fst stats2  NY_EU_un.fst.idx -win 10000 -step 2000 -type 1 > NY_EU_un.fst.sw

NJ Exposed vs Unexposed

In [ ]:
~/angsd/angsd/misc/realSFS fst index /local/home/sarahgw/WGS/SNPS/All_mylu/saf/NJ_E_unfilt.saf.idx ~/WGS/SNPS/All_mylu/saf/NJ_U_unfilt.saf.idx -sfs ~/WGS/SNPS/All_mylu/sfs/NJ_EU_unfilt_sum.ml -fstout NJ_EU_unfilt

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NJ_EU_unfilt.fst.idx

Fst.Weight:0.015852

In [ ]:
~/angsd/misc/realSFS fst stats2  NJ_EU_unfullnose.fst.idx -win 10000 -step 2000 -type 1 > NJ_EU_unfullnose.fst.sw

NJ Exposed vs Unexposed Dead

In [ ]:
~/angsd/angsd/misc/realSFS fst index ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/NJ_U_D_un.saf.idx -sfs ~/WGS/sfs/NJ_EUD_unfull_sum.ml -fstout NJ_EUD_un

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NJ_EUD_un.fst.idx

In [ ]:
~/angsd/angsd/misc/realSFS fst stats2  NJ_EUD_un.fst.idx -win 10000 -step 2000 -type 1 > NJ_EUD_un.fst.sw

VT Exposed vs Unexposed 

In [ ]:
~/angsd/misc/realSFS fst index ~/WGS/SNPS/All_mylu/saf/VT_E_un.saf.idx ~/WGS/SNPS/All_mylu/saf/VT_U_un.saf.idx -sfs /local/home/sarahgw/WGS/SNPS/All_mylu/sfs/VT_EU_unfullnose_sum.ml -fstout /local/home/sarahgw/WGS/SNPS/All_mylu/fst/VT_EU_unfullnose

In [ ]:
~/angsd/misc/realSFS fst stats2  VT_EU_unfullnose.fst.idx -win 10000 -step 2000 -type 1 > VT_EU_unfullnose.fst.sw

In [ ]:
~/angsd/misc/realSFS fst stats VT_EU_unfullnose.fst.idx

FST.Unweight[nObs:1733765270]:0.011765 Fst.Weight:0.014174

Calculate Three way fsts

Unexposed:

In [ ]:
~/angsd/angsd/misc/realSFS fst index /local/home/sarahgw/WGS/saf/NJ_U_un.saf.idx /local/home/sarahgw/WGS/saf/VT_U_un.saf.idx /local/home/sarahgw/WGS/saf/NY_U_un.saf.idx -sfs /local/home/sarahgw/WGS/sfs/NJVT_U_unfullnose_sum.ml -sfs /local/home/sarahgw/WGS/sfs/NJNY_U_unfullnose_sum.ml -sfs /local/home/sarahgw/WGS/sfs/VTNY_U_unfullnose_sum.ml -fstout /local/home/sarahgw/WGS/sfs/NJVTNY_U_unnose_reorder -P 30

In [ ]:
~/angsd/angsd/misc/realSFS fst stats ~/WGS/sfs/NJVTNY_U_unnose_reorder.fst.idx

        -> FST.Unweight[nObs:1694711106]:0.013878 Fst.Weight:0.017616
        
        -> FST.Unweight[nObs:1694711106]:0.014530 Fst.Weight:0.017590
        
        -> FST.Unweight[nObs:1694711106]:0.014752 Fst.Weight:0.017552

        -> pbs.pop1     0.008906
        -> pbs.pop2     0.008867
        -> pbs.pop3     0.008841

Exposed

In [ ]:
~/angsd/angsd/misc/realSFS fst index ~/WGS/saf/NY_E_un.saf.idx ~/WGS/saf/NJ_E_un.saf.idx ~/WGS/saf/VT_E_un.saf.idx -sfs ~/WGS/sfs/NJNY_E_unfull_sum.ml -sfs ~/WGS/sfs/VTNY_E_unfull_sum.ml -sfs ~/WGS/sfs/NJVT_E_unfull_sum.ml -fstout NJNYVT_E_un

In [ ]:
~/angsd/misc/realSFS fst stats ~/WGS/SNPS/All_mylu/fst/NJVTNY_E_un_reorder.fst.idx

      -> FST.Unweight[nObs:1694141456]:0.011623 Fst.Weight:0.014038
        -> FST.Unweight[nObs:1694141456]:0.013146 Fst.Weight:0.015207
        -> FST.Unweight[nObs:1694141456]:0.011074 Fst.Weight:0.013817
        -> pbs.pop1     0.007774
        -> pbs.pop2     0.006364
        -> pbs.pop3     0.007550

Outliers

In [ ]:
grep 'GL429767' -A 10000000 /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa | tr -d "\n\r" | cut -c 63839001-63865001 >/local/home/sarahgw/WGS/SNPS/All_mylu/snps/GL429767_63839001

In [ ]:
~/angsd/misc/realSFS fst print NJ_EU_unfull.fst.idx> NJ_EU_full_fst

In [ ]:
awk -F, 'FNR==NR {a[$1]=$0; next}; $1 in a {print a[$1]}' NJ_EU_full_FST NJ_SNP_sites1_sorted.txt

In [ ]:
~/angsd/misc/realSFS fst print VT_EU_full.fst.idx  > VT_EU_full_SNP_freq

In [ ]:
awk 'FNR==NR {a[$1$2]=$0; next}; $1$2 in a {print a[$1$2]}' VT_EU_full_SNP_freq VT_SNP_sites1_sorted.txt> VT_EU_full_SNP_fst

In [ ]:
~/angsd/misc/realSFS fst print NJ_EU_full.fst.idx > NJ_EU_full_SNP_freq

This seems to be arbitrarily filtering, the number of lines doesnt match and there are definitely sites in the output that aren't in the SNPs file, try again with a sorted SNP file? did't work. use awk to fiter by snps

In [ ]:
~/angsd/misc/realSFS fst print NJ_EU_full.fst.idx -sites NJ_SNP_sites1_sorted.txt > NJ_EU_full_SNP_freq1

In [ ]:
~/angsd/misc/realSFS fst print NY_EU_unfull.fst.idx -sites NY_SNP_sites1.txt > NY_EU_unfull_SNP_freq

In [ ]:
~/angsd/misc/realSFS fst print NY_EU_unfull.fst.idx> NJ_EU_unfull_fst

In [ ]:
~/angsd/misc/realSFS fst print VT_EU_full.fst.idx> VT_EU_full_fst

In [ ]:
grep 'AAPE02072785' NJ_EU_full_fst > NJ_AAPE02072785

This doesnt make much sense: mt change in allele freq: -0.0103409
full change in allele freq: -0.0153759

In [ ]:
awk '$4>0.40' /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_EU_full.fst.gz

Calculate FST for NY

Started at 1:15 PM on 7/11/17 on annotate

In [ ]:
~/angsd/angsd/misc/realSFS fst index NY_E_un.saf.idx NY_U_un.saf.idx -sfs NY_EU_unfull.ml -fstout NY_EU_unfull; echo "NY FST Done" | mailx gw.sarah@gmail.com

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NY_EU_unfull.fst.idx 

This is the unfolded FST values: -> FST.Unweight[nObs:1710526195]:0.031527 Fst.Weight:0.051177
This is the folded FST values:-> FST.Unweight[nObs:1710526195]:0.030941 Fst.Weight:0.050585

In [ ]:
~/angsd/misc/realSFS fst stats2 NY_EU_unfull.fst.idx -win 10000 -step 2000 -type 1 > NY_unfull_sw2

Calculate FST for VT

In [ ]:
~/angsd/angsd/misc/realSFS fst index VT_E.saf.idx VT_U.saf.idx -sfs VT_EU_full.ml -fstout VT_EU_full ; echo "VT FST Done"| mailx gw.sarah@gmail.com 

In [ ]:
~/angsd/angsd/misc/realSFS fst stats VT_EU_full.fst.idx 

-> FST.Unweight[nObs:1734759399]:0.029444 Fst.Weight:0.045089

In [ ]:
~/angsd/misc/realSFS fst stats2 VT_EU_full.fst.idx -win 10000 -step 2000 -type 1 > VT_full_sw2

In [ ]:
awk '$5>0.10' /local/home/sarahgw/WGS/SNPS/All_mylu/VT_sw 

Get sequences for AAPE02072785

In [ ]:
grep 'AAPE02072785' -A 300 /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa | tr -d "\n\r" | cut -c 7000-15000 >/local/home/sarahgw/WGS/SNPS/All_mylu/AAPE02072785

In [ ]:
52357001 52365001

In [ ]:
grep 'GL429767' -A 300000 /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa | tr -d "\n\r" | cut -c 52357001-52365001 >/local/home/sarahgw/WGS/SNPS/All_mylu/GL429767

Test for why GL429767 is no longer an FST outlier? 
Can I pull out the allele frequencies and rerun FST analysis with new allele freqs but only subset of samples?
Can I look to see if genotype likelihoods or allele frquencies change in this region with full dataset?
Can I test if the change is on the part of the Exposed or Unexposed populations?--first, try doing fst with saf of old nj exposed and new nj unexposed.  

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 angsdout_SAF_pairedplus_filt_E.saf.idx NJ_U.saf.idx > NJ_EU_sub.ml

In [ ]:
~/angsd/angsd/misc/realSFS fst index angsdout_SAF_pairedplus_filt_E.saf.idx NJ_U.saf.idx -sfs NJ_EU_sub.ml -fstout NJ_EU_sub

In [ ]:
~/angsd/angsd/misc/realSFS fst stats NJ_EU_sub.fst.idx 

FST.Unweight[nObs:1622446386]:0.046976 Fst.Weight:0.080651
Okay, so it seems like its not the exposed that are making a difference its the unexposed?

In [ ]:
~/angsd/angsd/misc/realSFS fst stats2 NJ_EU_sub.fst.idx -win 10000 -step 2000 -type 1 >  NJ_EU_sub_sw2